In [ ]:
"""
Q41 여행계획

즉 여행계획에 속한 노드들이 모두 포함되는 부분 그래프가 존재하는지는 확인하는 문제로 치환될 수 있다.
서로소 판별 알고리즘을 이용해서 같은 부모를 가진다면 같은 부분 그래프에 속한다고 판단할 수 있다.
"""

def find_parent(parent, node:int):
    """
    노드의 부모노드를 재귀적으로 탐색함. 만약 이미 부모 노드가 등록되어 있으면 
    """
    if parent[node] != node:
        parent[node] = find_parent(parent, parent[node])
    else:
        return parent[node]
    
    
def union(parent, node1, node2):
    """
    node1과 node2의 부모를 찾아서 통일함
    """
    a = find_parent(parent, node1)
    b = find_parent(parent, node2)
    
    # parent의 대소에 따라 큰 쪽이 작은 쪽을 가리키도록 함
    if a < b: #:
        parent[b] = a
    else:
        parent[a] = b
    return parent
        
N = int(input()) 
M = int(input())

# connectivity matrix 
connectivity = [list(map(int, input().split())) for _ in range(N)]

# 목표 서브그래프에 속한 노드 리스트
travel_plan = list(map(int, input().split()))

# 부모 노드 리스트 생성(최초 생성 시에는 자기 자신의 번호를 할당)
parent = list(range(1, N+1))

# 서로소 알고리즘 시작
for node1 in range(1, N+1):
    for node2 in range(1, N+1):
        if connectivity[node1-1][node2-1] == 1: # 간선이 있을 경우
            parent = union(parent, node1, node2)

# travel_plan 리스트에 있는 모든 노드가 같은 부모를 가지는지 확인
common_parent = parent[travel_plan[0]] # travel_plan에 속한 임의의 노드의 parent를 공통 부모로 간주
for node in travel_plan:
    if parent[node] != common_parent: # travel_plan에 속한 노드 중 하나라도 공통부모가 다르다면 NO를 출력하고 반복문 종료 
        print("NO")
        break
else:
    print("YES")

In [3]:
"""
Q42 탑승구

솔루션:
각 탑승구를 서로 다른 집합으로 표현하고, 루트 노드 번호 = 자기가 도킹하는 비행기 번호 와 동일시 한다. 
비행기가 도킹할 경우 가장 높은 노드의 번호로 도킹한다고 가정한다. 
0번 루트노드를 추가해 해당 노드를 최종적인 루트 노드로 가지는 경우 도킹이 불가함을 표시한다.

만약 도킹 시 가장 자기가 도킹 가능한 범위 내에서 가장 높은 번호의 루트를 확인해 
해당 노드가 만약 아직 도킹되지 않은 경우 (즉 어디와도 연결되지 않고 자기 자신을 여전히 루트로 가지고 있는 경우)
도킹한 노드들의 집합에 합연산을 하여 도킹 처리를 하고, 

만일 자리가 없는 경우 (자기가 도킹 가능한 범위 내에서 가장 높은 번호의 루트를 확인했을 때 최종적으로 0번 노드까지 연결되는 경우) 해당 비행기는 
도킹 처리를 하지 않는다. 

모든 비행기가 소진될 때까지 이를 반복한 후 도킹한 횟수를 반환한다. 

서로소집합을 찾는 알고리즘의 시간복잡도는 O(V + M(1 + Log{1-(m/v)}{V}))이다. 
V = 노드 (100,000)
M = find연산 개수 (100,000)
이므로 시간복잡도는 100,000 + 100,000(1 + 0)

엥 근데 log base 0면 결과값이 실수가 아니지 않나요?
"""

def find(parents, x):
    if parents[x] != x:
        parents[x] = find(parents, parents[x])
    return parents[x]

def union(parent, a, b):
    a = find(parents, a)
    b = find(parents, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b 


G = int(input()) # 탑승구 숫자
P = int(input()) # 비행기 숫자
parents = [i for i in range(G + 1)] # parents 배열 초기화
planes = [int(input()) for _ in range(P)]
result = 0 # 도킹 가능한 비행기 숫자 저장 

for p in planes: # 비행기 수 만큼 반복하면서
    current_station = find(parents, p) # 현재 항공기의 탑승구 루트 확인
    if current_station == 0: # 최종 루트가 0이면 탑승이 불가능하므로 
        break # 1대라도 도킹 못하면 중지해야 하므로 break
    print(parents)
    union(parents, current_station, current_station-1)
    result += 1

print(result)



[0, 1, 2, 3, 4]
[0, 1, 2, 3, 3]
2


In [37]:
"""
Q43 어두운 길

최소신장트리를 만드는 문제이다. 즉 모든 간선들을 비용 순으로 상향정렬한 후 하나씩 최소신장트리에 더하면서 모든 노드가 포함되는 순간 종료하면
이것이 현재 최소한의 비용을 가진 최소신장트리이다. 그리고 sum(전체 간선 비용) - sum(최소신장트리 간선 비용)을 하면 곧 절약할 수 있는 
최대 금액이다. 
"""

def find_parent(parent, node):
    if parent[node] != node:
        parent[node] = find_parent(parent, parent[node])
    return parent[node]
        

def union(parent, node1, node2):
    p1 = find_parent(parent, node1)
    p2 = find_parent(parent, node2)
    if p1 < p2:
        parent[p2] = p1
    else:
        parent[p1] = p2
    return parent

# 집,도로 개수 입력
house, road = map(int, input().split())

# 간선 입력
edges = [list(map (int, input().split())) for _ in range(road)]

# 간선을 비용 순서대로 상향 정렬
edges.sort(key=lambda x: x[2])

# 부모 노드 리스트 생성
parent = [i for i in range(house)]

# 최소신장트리의 총 비용 합
total = sum([i[2] for i in edges])
tree_total = 0

for edge in edges:
    node1, node2, cost = edge
    if find_parent(parent, node1) != find_parent(parent, node2): # 만약 두 노드의 부모가 다르다면 사이클 x 
        parent = union(parent, node1, node2)
        tree_total += cost
        
print(total - tree_total)

51


In [12]:
"""
Q44 행성 터널
https://www.acmicpc.net/problem/2887

N개의 행성들 가운데 N-1개의 통로를 설치하여 모든 행성을 연결해야 한다는 점에서 크루스칼 알고리즘을 통한 최소신장트리를 만드는 문제로 표면적으로는 보이나
모든 거리를 연산하면 시간 초과가 뜬다. 

핵심은 크루스칼을 사용하되 비용을 줄이는 것인데, 두 행성 간 비용을 연산하는 공식이 min(x축상에서의 거리, y축상에서의 거리, z축상에서의 거리) 이므로
x, y, z를 따로 분해한 후 각각 각자의 축 상에서 다른 모든 행성 간의 거리를 계산하는 것이 아니라 정렬 후 "자기 축 상에서 가장 가까운 이웃 하나"와의 거리만을 구하는 식으로
거리를 구하는 횟수를 감소시키는 것이다. 

그 후 연산한 모든 거리들을 한 리스트에 넣은 후, 정렬한 다음에 "cycle이 발생하지 않는 선"에서 순서대로 다 합하면 결과값은 최소신장트리의 비용합이 된다. 
"""
def find(parents, x):
    if parents[x] != x:
        parents[x] = find(parents, parents[x])
    return parents[x]

def union(parents, x, y):
    x = find(parents, x)
    y = find(parents, y)
    if x < y:
        parents[y] = x
    else: 
        parents[x] = y

n = int(input())
parents = [i for i in range(n + 1)]


# 모든 간선을 담을 리스트와 최종비용을 담은 변수 생성
edges = []
result = 0

# 각 행성의 x/y/z축 선상에서의 거리를 담는 리스트 생성
x = []
y = [] 
z = []

for i in range(1, n + 1):
    row = list(map(int, input().split())) # (x, y, z) 형식으로 한 행성의 정보가 들어오므로 분해하여 (좌표, 번호) 형식으로 상응하는 리스트에 append
    x.append((row[0], i))
    y.append((row[1], i))
    z.append((row[2], i))

x.sort() 
y.sort()
z.sort()

for i in range(n - 1):
    # 각각의 축상에서의 거리비용을 계산한 후 이를 모두 edges에 넣은 후 상향정렬한다. 
    edges.append((x[i + 1][0] - x[i][0], x[i][1], x[i + 1][1])) # (i번 노드와 i+1번 노드와의 x축 상에서의 거리, i, i+1)
    edges.append((y[i + 1][0] - y[i][0], y[i][1], y[i + 1][1]))
    edges.append((z[i + 1][0] - z[i][0], z[i][1], z[i + 1][1]))

edges.sort()


for edge in edges:
    cost, a, b = edge
    if find(parents, a) != find(parents, b):
        union(parents, a, b)
        result += cost

print(f"result={result}")
print("=" * 50)
print(f"edges={edges}")
print("=" * 50)
print(x)
print("=" * 50)
print(y)
print("=" * 50)
print(z)


edges=4
edges=[(0, 1, 2), (0, 4, 5), (1, 2, 4), (1, 4, 1), (3, 1, 2), (3, 5, 3), (4, 3, 4), (5, 2, 5), (10, 1, 2), (10, 2, 3), (11, 3, 4), (20, 4, 5)]
[(-1, 3), (10, 4), (11, 1), (14, 2), (19, 5)]
[(-15, 1), (-5, 2), (-4, 4), (-4, 5), (-1, 3)]
[(-15, 1), (-15, 2), (-5, 3), (-1, 4), (19, 5)]


In [13]:
result

4

In [ ]:
"""
Q45 최종 순위
https://www.acmicpc.net/problem/3665

순위 변동이 없는 팀은 어차피 등수를 알 수 있고, 변동이 있는 팀들을 모은 부분집합에서 위상 정렬을 할 수 있다면 순위 변동이 없는 팀과 조합해서 모든 순위를 알 수 있다.
일반적인 위상 정렬을 하되, 두 가지 경우를 고려한다. 
1. 조기 종료되는 경우 (순위는 1방향인데 Cyclicity가 있을 수가 없다. 이럴 경우 조기종료한다)
2. 가능한 순위가 1개 이상 있을 경우 -> 이 경우에도 실제 순위를 알 수 없으므로 추가적인 탐색의 의미가 없다. 

즉 위상 정렬을 수행하되 queue에서 원소를 꺼낼 때마다 1번과 2번 조건에 해당되는지 확인하고 만약 해당된다면 조기종료하고 IMPOSSIBLE 반환, 그렇지 않고
정상적으로 모든 queue가 소모된다면 ?을 반환하면 된다.

2번의 경우, 위상 정렬에서 
""" 

from collections import deque

test = int(input())
for i in range(test):
    n = int(input()) # 팀 개수
    graph = [[False] * (n + 1) for i in range(n + 1)] # 
    indegree = [0 for _ in range(n + 1)] # 진입 차수 리스트 생성

    # 작년 순위 리스트 저장
    prev_rank = list(map(int, input().split())) 

    # 작년 순위 그래프 구성함
    for i in range(n): 
        for j in range(i+1, n): 
            graph[prev_rank[i]][prev_rank[j]] = True
            indegree[prev_rank[j]] += 1 # 순위가 낮은 팀 번호를 가리키게 되므로 a개의 노드가 있을 때 임의의 노드 i에 대해 노드 (i < j <= a) 들에 진입간선이 생김.

    
    # 변동이 있는 팀 숫자 
    # 기존에 노드 a의 순위 > 노드 b일 때, graph에선 graph[a][b] = True 였는데 관계가 역전됐으니 간선의 방향을 바꿔야 하므로 graph[b][a] = True를 놓고 나머지 정보도 반영한다
    changed = int(input())
    for i in range(changed):
        a, b = map(int, input().split()) 
        
        if graph[a][b]:
            graph[b][a] = True
            graph[a][b] = False
            indegree[b] -= 1
            indegree[a] += 1
        else:
            graph[b][a] = False
            graph[a][b] = True
            indegree[b] += 1
            indegree[a] -= 1

    queue = deque()

    # 진입차수가 0인 노드들을 queue에 넣음
    for i in range(1, n + 1):
        if indegree[i] == 0:
            queue.append(i)
    
    # sorting 시작
    run = 0 # queue의 시행 횟수 카운트용
    result = [] # 정렬된 순위
    unknown = False # 순위 불투명 여부
    cyclic = False # cyclicty 여부

    while queue:  
        new = 0 # 새로 진입차수가 0이 된 노드 카운트용
        node = queue.popleft()
        run += 1
        result.append(node)

        # 현재 node에서 출발하는 간선들 제거
        for next_node in range(1, n + 1):
            if graph[next_node]: # 현재노드와 연결이 되어 있는 노드는
                indegree[next_node] -= 1 # 진입차수 -1 
                if indegree[next_node] == 0: # 이로 인해 진입차수가 0이 된다면:
                    new += 1
                    queue.append(next_node)
        
        # 만약 새롭게 추가되는 노드 수가 2개 이상일 경우 순위를 알 수가 없다. 
        if new >= 2: 
            unknown = True 
            break

    # unknown 여부가 False인 상태에서 run이 n보다 작다면 (조기종료됐다면) cyclicity가 있다. 
    if (run < n) and (unknown == False):
        cyclic = True-15
        14
        

    # 답 출력 
    if cyclic:
        print("IMPOSSIBLE")
    
    elif unknown:
        print("?")
    
    else:
        for i in result:
            print(i, end = " ")
        print()